In [1]:
import numpy as np
import torch

# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

# sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference import SNLE, prepare_for_sbi, simulate_for_sbi
from sbi.inference.base import infer

/Users/ryankelly/python_projects/sbi/venv/lib/python3.9/site-packages/tqdm-4.64.0-py3.9.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def MG1(params, n_obs=50, batch_size=1, random_state=None):
    """Generate a sequence of samples from the M/G/1 model.

    The sequence is a moving average

    Parameters
    ----------
    t1 : float, array_like
        minimum service time length
    t2 : float, array_like
        maximum service time length
    t3 : float, array_like
        Time between arrivals Exp(t3) distributed
    n_obs : int, optional
    batch_size : int, optional
    random_state : RandomState, optional

    """
    if isinstance(params, torch.Tensor):
        t1, t2, t3 = float(params[0][0]), float(params[0][1]), float(params[0][2])
    else:
        t1, t2, t3 = params[0], params[1], params[2]

    # if hasattr(t1, 'shape'):  # assumes vector consists of identical values
    #     t1, t2, t3 = t1[0], t2[0], t3[0]

    random_state = random_state or np.random

    # arrival time of customer j after customer j - 1
    W = random_state.exponential(1/t3, size=(batch_size, n_obs))  # beta = 1/lmda
    # service times
    U = random_state.uniform(t1, t2, size=(batch_size, n_obs))

    y = np.zeros((batch_size, n_obs))
    sum_w = W[:, 0]  # arrival time of jth customer, init first time point
    sum_x = np.zeros(batch_size)  # departure time of the prev customer, init 0s

    for i in range(n_obs):
        y[:, i] = U[:, i].flatten() + np.maximum(np.zeros(batch_size), sum_w - sum_x).flatten()
        sum_w += W[:, i]
        sum_x += y[:, i-1]

    return y


In [3]:
def simulation_wrapper(params):
    x_sim = MG1(params)
    sim_sum = torch.as_tensor(x_sim.astype('float32'))
    return sim_sum.reshape((-1, 50))  # TODO: magic number 50

In [4]:
# TODO: ACTUAL PRIOR?
prior_min = [0, 0, 0 ]
prior_max = [10, 10, 0.5]
prior = utils.torchutils.BoxUniform(low=torch.as_tensor(prior_min),
                                    high=torch.as_tensor(prior_max))

In [5]:
inference = SNLE(prior=prior)

In [7]:
true_params = [1., 5., 0.2]
y = MG1(true_params)

num_rounds = 10

posteriors = []
proposal = prior

for _ in range(num_rounds):
    theta, x = simulate_for_sbi(simulation_wrapper, proposal, num_simulations=300)
    density_estimator = inference.append_simulations(theta, x
    ).train()
    posterior = inference.build_posterior(density_estimator)
    posteriors.append(posterior)
    proposal = posterior.set_default_x(y)


Running 300 simulations.: 100%|██████████| 300/300 [00:00<00:00, 1062.95it/s]


 Neural network successfully converged after 254 epochs.

Generating samples:  27%|██▋       | 82/300 [00:35<01:33,  2.34it/s]


KeyboardInterrupt: 

In [ ]:
samples = posterior.sample((5000,),
                           x=y)

In [ ]:
fig, axes = analysis.pairplot(samples,
                           limits=[[-1, 1], [-1, 1]],
                        #    ticks=[[.5, 1], [.5, 15.]],
                           figsize=(5,5),
                        #    points=true_params,
                           points_offdiag={'markersize': 6},
                           points_colors='r');